In [4]:
import { SupabaseVectorStore } from "@langchain/community/vectorstores/supabase";
import { OpenAIEmbeddings } from "@langchain/openai";
import { createClient } from "@supabase/supabase-js";
import dotenv from "dotenv";
dotenv.config();

const embeddings = new OpenAIEmbeddings({
    model: "text-embedding-3-small",
  });

  const supabaseClient = createClient(
    process.env.SUPABASE_URL ?? '',
    process.env.SUPABASE_SERVICE_ROLE_KEY ?? '',
  );
  const vectorStore = new SupabaseVectorStore(embeddings, {
    client: supabaseClient,
    tableName: 'documents',
    queryName: 'match_documents',
  });



In [5]:
import { PDFLoader } from "@langchain/community/document_loaders/fs/pdf";
import { RecursiveCharacterTextSplitter } from "@langchain/textsplitters";
import { writeFile } from "fs/promises";
import { reduceDocs } from "./src/shared/state.js";

const pdfPath = "./test_docs/test-tsla-10k-2023.pdf"

const loader = new PDFLoader(pdfPath);

const docs = await loader.load();


const textSplitter = new RecursiveCharacterTextSplitter({
    chunkSize: 2000,
    chunkOverlap: 50,
  });
const docSplits = await textSplitter.splitDocuments(docs);

const finalDocs = reduceDocs([], docSplits);

  
finalDocs[0]



{
  pageContent: 'UNITED\tSTATES\n' +
    'SECURITIES\tAND\tEXCHANGE\tCOMMISSION\n' +
    'Washington,\tD.C.\t20549\n' +
    'FORM\t10-K\n' +
    '(Mark\tOne)\n' +
    'xANNUAL\tREPORT\tPURSUANT\tTO\tSECTION\t13\tOR\t15(d)\tOF\tTHE\tSECURITIES\tEXCHANGE\tACT\tOF\t1934\n' +
    'For\tthe\tfiscal\tyear\tended\tDecember\t31,\t2023\n' +
    'OR\n' +
    'oTRANSITION\tREPORT\tPURSUANT\tTO\tSECTION\t13\tOR\t15(d)\tOF\tTHE\tSECURITIES\tEXCHANGE\tACT\tOF\t1934\n' +
    'For\tthe\ttransition\tperiod\tfrom\t_________\tto\t_________\n' +
    'Commission\tFile\tNumber:\t001-34756\n' +
    'Tesla,\tInc.\n' +
    '(Exact\tname\tof\tregistrant\tas\tspecified\tin\tits\tcharter)\n' +
    'Delaware91-2197729\n' +
    '(State\tor\tother\tjurisdiction\tof\n' +
    'incorporation\tor\torganization)\n' +
    '(I.R.S.\tEmployer\n' +
    'Identification\tNo.)\n' +
    '1\tTesla\tRoad\n' +
    'Austin,\tTexas78725\n' +
    '(Address\tof\tprincipal\texecutive\toffices)(Zip\tCode)\n' +
    '(512)\t516-8177\n' +


In [13]:
const documentIds = finalDocs.map((doc) => doc.id);
await vectorStore.addDocuments(finalDocs, {ids: documentIds});
  

[
  'f96c76fd-d49a-476e-8565-29bbe5d60101',
  'cf036396-4b2b-46fb-9ca4-20bbd6a3947e',
  'f725b551-cdad-4b17-9681-6c7ac07742aa',
  '950dc7ea-3040-4e32-926f-3d2ad76598ba',
  'd756834a-3b9c-4ac4-90f2-5a5bb9a88b89',
  '15dcb874-9a11-414c-a612-4ba9ac78df9b',
  '27c14e27-544b-4e2d-bd30-5271fc1c1945',
  'c087b995-9b3c-43c9-849e-1ffccfa710e0',
  'ac0b84ff-f893-4a95-aaad-c79b9951c32e',
  '2f176e4f-1128-47ee-8803-ee004b725458',
  'b72bf8bd-47be-4392-8781-1fee00cd8028',
  'f87b847c-07e7-4e87-8a73-533028f7d778',
  'a2a571cf-bbec-40b6-a5e7-8a33fd36d1dd',
  '2405ac0e-8041-41e6-b8d2-c695a03a8a9a',
  '157becfc-79c8-43f6-941b-df7e62a51811',
  '79d2e706-1212-4a89-85f6-ff117f8a43cc',
  '97353508-c3d5-4c16-9b13-ab792aabd89b',
  'c63472f4-bd4d-4469-96a5-8eb01cd5fad2',
  '0f221405-317e-4c2a-ac61-7ad049a3bcf4',
  'de3767fe-d129-4218-b062-4b3c689797a6',
  'ba25d45b-6d25-467e-bf25-0106088d9da8',
  '4a391f87-1ccb-4f66-8cc8-d7a7f98c1455',
  '844ef3dc-9281-4912-9ddb-52260891237d',
  'c9b88bc9-6711-4c1d-a44d-878f3

In [14]:
//count the number of documents in the vector store

await vectorStore.client.from('documents').select('id', { count: 
'exact'});


{
  error: null,
  data: [
    { id: 'f96c76fd-d49a-476e-8565-29bbe5d60101' },
    { id: 'cf036396-4b2b-46fb-9ca4-20bbd6a3947e' },
    { id: 'f725b551-cdad-4b17-9681-6c7ac07742aa' },
    { id: '950dc7ea-3040-4e32-926f-3d2ad76598ba' },
    { id: 'd756834a-3b9c-4ac4-90f2-5a5bb9a88b89' },
    { id: '15dcb874-9a11-414c-a612-4ba9ac78df9b' },
    { id: '27c14e27-544b-4e2d-bd30-5271fc1c1945' },
    { id: 'c087b995-9b3c-43c9-849e-1ffccfa710e0' },
    { id: 'ac0b84ff-f893-4a95-aaad-c79b9951c32e' },
    { id: '2f176e4f-1128-47ee-8803-ee004b725458' },
    { id: 'b72bf8bd-47be-4392-8781-1fee00cd8028' },
    { id: 'f87b847c-07e7-4e87-8a73-533028f7d778' },
    { id: 'a2a571cf-bbec-40b6-a5e7-8a33fd36d1dd' },
    { id: '2405ac0e-8041-41e6-b8d2-c695a03a8a9a' },
    { id: '157becfc-79c8-43f6-941b-df7e62a51811' },
    { id: '79d2e706-1212-4a89-85f6-ff117f8a43cc' },
    { id: '97353508-c3d5-4c16-9b13-ab792aabd89b' },
    { id: 'a59d30c4-4ff6-4ade-b7b7-72130510fdc2' },
    { id: 'c63472f4-bd4d-4469-96a5-8e

In [15]:
await vectorStore.similaritySearch("risk factors");

[
  Document {
    pageContent: 'security\tincidents,\twhich\tcould\tresult\tin\tdata\tbreaches,\tintellectual\tproperty\ttheft,\tclaims,\tlitigation,\tregulatory\tinvestigations,\tsignificant\tliability,\n' +
      'reputational\tdamage\tand\tother\tadverse\tconsequences”\tincluded\tas\tpart\tof\tour\trisk\tfactor\tdisclosures\tat\tItem\t1A\tof\tthis\tAnnual\tReport\ton\tForm\t10-K.\n' +
      'Cybersecurity\tGovernance\n' +
      'Cybersecurity\tis\tan\timportant\tpart\tof\tour\trisk\tmanagement\tprocesses\tand\tan\tarea\tof\tfocus\tfor\tour\tBoard\tand\tmanagement.\tOur\tAudit\tCommittee\tis\n' +
      'responsible\tfor\tthe\toversight\tof\trisks\tfrom\tcybersecurity\tthreats.\tMembers\tof\tthe\tAudit\tCommittee\treceive\tupdates\ton\ta\tquarterly\tbasis\tfrom\tsenior\n' +
      'management,\tincluding\tleaders\tfrom\tour\tInformation\tSecurity,\tProduct\tSecurity,\tCompliance\tand\tLegal\tteams\tregarding\tmatters\tof\tcybersecurity.\tThis\n' +
      'includes\texisting\tand\tnew\t

In [16]:
import { pull } from "langchain/hub";
import { ChatPromptTemplate } from "@langchain/core/prompts";
import { z } from "zod";
import { ChatOpenAI } from "@langchain/openai";

const retriever = vectorStore.asRetriever({
  k: 2,
})

const answerSchema = z.object({
  answer: z.string().describe("The answer to the question"),
  sources: z.array(z.string()).describe("The full text of the documents used to answer the question"),
});

const llm = new ChatOpenAI({
    model: "gpt-4o",
    temperature: 0,
  })


const promptTemplate = await pull<ChatPromptTemplate>("rlm/rag-prompt");

const query = "What is the name of the company in the report?"

const retrievedDocs = await retriever.invoke(query)

const formattedPrompt = await promptTemplate.invoke({
  context: retrievedDocs,
  question: query,
});

const response = await llm.invoke(formattedPrompt);

console.log(response);


AIMessage {
  "id": "chatcmpl-AvntuDz8xFKGEO0DZMK5vBU0xcFZL",
  "content": "The name of the company in the report is Tesla, Inc.",
  "additional_kwargs": {},
  "response_metadata": {
    "tokenUsage": {
      "promptTokens": 1653,
      "completionTokens": 14,
      "totalTokens": 1667
    },
    "finish_reason": "stop",
    "model_name": "gpt-4o-2024-08-06",
    "usage": {
      "prompt_tokens": 1653,
      "completion_tokens": 14,
      "total_tokens": 1667,
      "prompt_tokens_details": {
        "cached_tokens": 0,
        "audio_tokens": 0
      },
      "completion_tokens_details": {
        "reasoning_tokens": 0,
        "audio_tokens": 0,
        "accepted_prediction_tokens": 0,
        "rejected_prediction_tokens": 0
      }
    },
    "system_fingerprint": "fp_50cad350e4"
  },
  "tool_calls": [],
  "invalid_tool_calls": [],
  "usage_metadata": {
    "output_tokens": 14,
    "input_tokens": 1653,
    "total_tokens": 1667,
    "input_token_details": {
      "audio": 0,
      "

In [17]:
// delete all documents from the vector store

await vectorStore.client.from('documents').delete().neq('id', '');
    

{
  error: {
    code: '22P02',
    details: null,
    hint: null,
    message: 'invalid input syntax for type uuid: ""'
  },
  data: null,
  count: null,
  status: 400,
  statusText: 'Bad Request'
}


UncaughtException: Error: Unexpected pending rebuildTimer
    at sys.setTimeout (/opt/homebrew/lib/node_modules/tslab/dist/converter.js:111:19)
    at Object.scheduleInvalidateResolutionsOfFailedLookupLocations (/opt/homebrew/lib/node_modules/tslab/node_modules/@tslab/typescript-for-tslab/lib/typescript.js:122719:55)
    at scheduleInvalidateResolutionOfFailedLookupLocation (/opt/homebrew/lib/node_modules/tslab/node_modules/@tslab/typescript-for-tslab/lib/typescript.js:121553:22)
    at /opt/homebrew/lib/node_modules/tslab/node_modules/@tslab/typescript-for-tslab/lib/typescript.js:121474:9
    at /opt/homebrew/lib/node_modules/tslab/node_modules/@tslab/typescript-for-tslab/lib/typescript.js:5810:11
    at /opt/homebrew/lib/node_modules/tslab/node_modules/@tslab/typescript-for-tslab/lib/typescript.js:5560:101
    at Array.forEach (<anonymous>)
    at /opt/homebrew/lib/node_modules/tslab/node_modules/@tslab/typescript-for-tslab/lib/typescript.js:5560:85
    at FSWatcher.callbackChangingT